In [1]:
!git clone  https://<username>:<token>@github.com/TeamEpochGithub/kenyan-sign-language.git
%cd kenyan-sign-language
!git submodule update --init

fatal: destination path 'kenyan-sign-language' already exists and is not an empty directory.
/content/kenyan-sign-language


In [2]:
!pip3 install tensorflow-addons --quiet

In [3]:
import sys
sys.path.append("models/tensorflow/automl/efficientnetv2")

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from src.preprocessing.tf_dataset_parsers import TfLabler
from src.preprocessing.tf_dataset_parsers import TfSubmiter
from src.preprocessing.tf_dataset_parsers import TfPresenter
from src.models.efficentnetV2 import EffiecntHead
from src.helpers import count_ds
from src.models.land import LandmarkHead

from src.preprocessing.runtime_preprocessors import randaugment_with_chioce
from src.tpu import connect
from src.tf_submission import generator

from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.optimizers.schedules import CosineDecay

import tensorflow_addons as tfa
from tensorflow_addons.optimizers import SGDW

from effnetv2_model import get_model

from src.models.warmup import WarmupSchedule
import random
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

In [5]:
def normalize(image):
  image = tf.cast(image,tf.float32)/127.5
  image = image - 1
  return image


available_ops = [
    'AutoContrast', 'Equalize', 'Invert', 'Rotate', 'Posterize',
    'Solarize', 'Color', 'Contrast', 'Brightness', 'Sharpness',
    'ShearX', 'ShearY', 'TranslateX', 'TranslateY', 'Cutout', 'SolarizeAdd']
augmenter = lambda x: randaugment_with_chioce(x,5,7,available_ops)

In [6]:
strategy = connect()

Running on TPU  ['10.22.32.170:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.22.32.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.32.170:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


# Experiment 1

In [7]:
# Try 19
# [I 2022-02-07 17:32:17,782] Trial 19 finished with value: 0.19401868879795076 and parameters: {'lr': 0.42747552824846474, 'lr_back': 0.2957642320558607, 'momentum': 0.42250027093428355, 'nesterov': True, 'weght_decay': 3.8052651581643845e-05, 'epochs': 8, 'layers': 5, 'strength': 7, 'alpha': 0.6346375572327532}. Best is trial 19 with value: 0.19401868879795076.

In [8]:
def make_cos(lr,count,epochs,alpha):
  mainsched = tf.keras.optimizers.schedules.CosineDecay(
        lr, count*epochs, name=None, alpha=alpha
      )
  warmuped = WarmupSchedule(count*3, mainsched)

  return warmuped
def init_model(count):
  tf.keras.backend.clear_session()
  with strategy.scope():
    eff_back = get_model(model_name='efficientnetv2-m',weights=None,include_top=False, training=True)
    pretrained_ckpt = tf.train.latest_checkpoint('gs://chimps-first/efficientnetv2-m-21k')
    eff_back.load_weights(pretrained_ckpt)
    model = LandmarkHead(eff_back)

    momentum = 0.42
    wd = 3.8e-05
    nest = True
    epochs = 18
    alpha = 0.635

    scheduer_head = make_cos(0.427,count,epochs,alpha)
    sgd_head = SGDW(learning_rate=scheduer_head,momentum=momentum,weight_decay=wd,nesterov=nest)
    scheduer_back = make_cos(0.126,count,epochs,alpha)
    sgd_back = SGDW(learning_rate=scheduer_back,momentum=momentum,weight_decay=wd,nesterov=nest)
    model.compute_output_shape(input_shape=(None,480,480,3))
    optimizers_and_layers = [(sgd_back, model.back), (sgd_head, model.head)]
    optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)
    model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy())

  return model

In [9]:
parser = TfLabler(processors=[augmenter,normalize])
alltfr = tf.data.TFRecordDataset("gs://chimps-first/data/signs.tfrecords")
all = alltfr.map(parser, num_parallel_calls=tf.data.AUTOTUNE).shuffle(128, reshuffle_each_iteration=True).batch(64,drop_remainder=True).prefetch(4)
count = alltfr.batch(64,drop_remainder=True)
count = count_ds(count)

# Trainer

In [10]:
model = init_model(count)
history = model.fit(
  all, epochs=21,verbose=True
)

Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


Epoch 1/21
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


97/97 [==============================] - 212s 294ms/step - loss: 2.1752
Epoch 2/21
97/97 [==============================] - 32s 291ms/step - loss: 0.8852
Epoch 3/21
97/97 [==============================] - 32s 290ms/step - loss: 0.5807
Epoch 4/21
97/97 [==============================] - 32s 290ms/step - loss: 0.4117
Epoch 5/21
97/97 [==============================] - 32s 293ms/step - loss: 0.2840
Epoch 6/21
97/97 [==============================] - 32s 290ms/step - loss: 0.2142
Epoch 7/21
97/97 [==============================] - 33s 303ms/step - loss: 0.1788
Epoch 8/21
97/97 [==============================] - 32s 293ms/step - loss: 0.1243
Epoch 9/21
97/97 [==============================] - 32s 289ms/step - loss: 0.0945
Epoch 10/21
97/97 [==============================] - 32s 289ms/step - loss: 0.0965
Epoch 11/21
97/97 [==============================] - 32s 289ms/step - loss: 0.0649
Epoch 12/21
97/97 [==============================] - 32s 289ms/step - loss: 0.0679
Epoch 13/21
97/97 [====

# Submit

In [11]:
authtoken = "KHkJpqasteMDQGYAAgaBM2xe"
!wget --post-data="auth_token=$authtoken" https://api.zindi.africa/v1/competitions/kenyan-sign-language-classification-challenge/files/SampleSubmission.csv -q

In [12]:
subcs = pd.read_csv("SampleSubmission.csv",index_col=0)
subcs.head(3)

,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
img_IDS,,,,,,,,,
ImageID_USRB8QNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ImageID_SZ8D1ZJI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ImageID_4OJO2F8J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
sumbm = TfSubmiter(processors=[normalize])
submit = tf.data.TFRecordDataset("gs://chimps-first/data/tosubmit.tfrecords").map(sumbm).batch(64)
subcs = generator(submit,model,subcs)
subcs.head(4)

,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
img_IDS,,,,,,,,,
ImageID_USRB8QNG,0.000005,0.000004,0.000009,2.711812e-08,2.815764e-06,0.000004,4.953059e-08,0.999973,2.060387e-06
ImageID_SZ8D1ZJI,0.000005,0.999697,0.000073,1.210710e-05,1.323906e-05,0.000090,1.419947e-07,0.000096,1.224312e-05
ImageID_4OJO2F8J,0.000131,0.000036,0.000007,2.227589e-05,6.722541e-07,0.999802,7.180308e-07,0.000001,2.875655e-07
ImageID_IEE4XV0B,0.000020,0.000002,0.000003,6.295171e-06,6.918470e-06,0.999873,3.405277e-05,0.000054,3.916280e-07


In [14]:
subcs.to_csv("eff_long_subm.csv")
from google.colab import files
files.download('eff_long_subm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>